In [1]:
import os
import json
import numpy
import pyccl
import scipy

In [2]:
# FOLDER
FOLDER = '/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

In [3]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY = json.load(file)

CCL = pyccl.Cosmology(
    h = COSMOLOGY['H'],
    w0 = COSMOLOGY['W0'],
    wa = COSMOLOGY['WA'], 
    n_s = COSMOLOGY['NS'], 
    A_s = COSMOLOGY['AS'],
    m_nu = COSMOLOGY['M_NU'],  
    Neff = COSMOLOGY['N_EFF'],
    Omega_b = COSMOLOGY['OMEGA_B'], 
    Omega_k = COSMOLOGY['OMEGA_K'], 
    Omega_c = COSMOLOGY['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [4]:
# Comparison
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=CCL, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=CCL, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=CCL, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

In [5]:
# Numerical integral

def integral_I10(chi1, chi2, chi3, chi4, power1, power2, redshift1, redshift2):
    
    def formula_0(chi):
        
        result = (- chi + (chi4 - chi3) / 2 + chi * chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3)) ** 2
        
        result = result * (((chi/chi2) ** 3)*power2)
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    def formula_n(chi):
        result = (- chi + (chi4 - chi3) / 2 + chi * chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3)) ** 2
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

In [6]:
# Coefficient

def coefficient_J10(chi1, chi2, chi3, chi4, power1, power2, redshift1, redshift2):
    a = 1 - chi1 / chi2
    b = (chi4 - chi3) / (2 * chi2)
    c = chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3) - 1
    p = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)
    
    if a == 1:
        formula = (1 / 840) * (5 * c ** 2 * (28 + ( - 8 + z) * z) + 16 * b * c * (21 + ( - 7 + z) * z) + 14 * b ** 2 * (15 + ( - 6 + z) * z))
    else:
        formula = (1 / 60) * a * (c ** 2 * (20 * (3 + ( - 3 + a) * a) - 5 * (6 + a * ( - 8 + 3 * a)) * p + 2 * ( - 30 + 5 * (8 - 3 * a) * a + 20 * p + 6 * a * ( - 5 + 2 * a) * p) * z + 
(20 - 15 * p + 2 * a * ( - 15 + 6 * a + 12 * p - 5 * a * p)) * z ** 2) + 2 * b * c * (20 * (3 + ( - 3 + z) * z) + a * ( - 30 + 20 * p + 5 * (8 - 3 * z) * z + 6 * p * z * ( - 5 + 2 * z)) - 
5 * p * (6 + z * ( - 8 + 3 * z))) - 5 * b ** 2 * ( - 4 * (3 + ( - 3 + z) * z) + p * (6 + z * ( - 8 + 3 * z))))
    
    coefficient = chi2 ** 3 * power2 * (1 + redshift2) ** 2 * formula
    return coefficient

In [7]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = numpy.float128(CHI_GRID[GRID_INDEX])
CHI2 = numpy.float128(CHI_GRID[GRID_INDEX + 1])
CHI3 = numpy.float128(CHI_GRID[GRID_SIZE - 1])
CHI4 = numpy.float128(CHI_GRID[GRID_SIZE])

POWER1 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX])
POWER2 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX + 1])

REDSHIFT1 = numpy.float128(Z_GRID[GRID_INDEX])
REDSHIFT2 = numpy.float128(Z_GRID[GRID_INDEX + 1])

print(CHI1, CHI2, CHI3, CHI4, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

4924.221744025123371 4940.716645699534638 6946.038887321783477 6954.2547327845313703 11370.147631619063759 11268.969970366384587 1.75 1.7600000000000000089


In [8]:
# Case1: chi1 > 0

INTEGRAL = integral_I10(CHI1, CHI2, CHI3, CHI4, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J10(CHI1, CHI2, CHI3, CHI4, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

2017446.8291593744019 2017446.8291593735556 -4.1947782053464166552e-16


In [9]:
# Case2: chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_SIZE - 1]
CHI4 = CHI_GRID[GRID_SIZE]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3,CHI4,POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.40057919571794 6946.0388873217835 6954.254732784531 0.0 1826.8838600584127 0.0 0.01


In [10]:
# Case2: chi1 = 0

INTEGRAL = integral_I10(CHI1, CHI2, CHI3, CHI4, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J10(CHI1, CHI2, CHI3, CHI4, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

344155.0938873242 344155.0938873238 -1.2212453270876722e-15
